### HMMSCAN Parser

In [10]:
from collections import defaultdict
import pandas as pd
from Bio import SearchIO

filename = 'Test2.txt'

attribs = ['query_id', 'bitscore', 'evalue', 'id']

hits = defaultdict(list)

with open (filename) as handle:
    for record in SearchIO.parse(handle, 'hmmer3-tab'):
      #print(queryresult.id)
      #print(queryresult.accession)
      #print(queryresult.description)
        for hit in record.hits:
            for attrib in attribs:
                hits[attrib].append(getattr(hit, attrib))

df = pd.DataFrame.from_dict(hits)
df.head()

,query_id,bitscore,evalue,id
0,Bacteria_NODE_1_length_628658_cov_8.291329_24,29.3,7.100000e-07,3300000568@Draft_10015026@Draft_1001502652
1,Bacteria_NODE_1_length_628658_cov_8.291329_53,31.7,1.600000e-07,7000000546@SRS019910_WUGC_scaffold_3948@SRS019...
2,Bacteria_NODE_1_length_628658_cov_8.291329_54,51.0,1.500000e-13,ADO97609@GU071095
3,Bacteria_NODE_1_length_628658_cov_8.291329_93,44.6,1.600000e-11,3300000101@DelMOSum2010_c10000158@DelMOSum2010...
4,Bacteria_NODE_1_length_628658_cov_8.291329_120,33.0,7.000000e-08,3300001592@Draft_10000123@Draft_1000012318


### Diamond Blast Parser

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
input_file = "../Earth_Virome/Earth_Virome_Data/Uniform_Low_Complexity.txt"
#input_file = "../../Nano_Reads_Tools/Nano_Combined_Assemblies_Diamond.txt"
df = pd.read_csv(input_file, header=None, sep="\t")


df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")
simulation = input_file.split("_")[-3].split("/")[1]

if simulation == 'Zero':
    complexity = input_file.split("_")[-2]
else:
    complexity = input_file.split("_")[-2]


df1 = df2[(df2.Simulation == simulation) & (df2.Complexity == complexity)]
try:
    Phage_Count = int(df1['Phage'])
except:
    Phage_Count = 0
try:
    Prophage_Count = int(df1['Prophage'])
except:
    Prophage_Count = 0
    
 
   
#Phage_Count=2514
#Phage_Count=34

headers = ['qseqid', 'sseqid','pident','length', 'mismatch', 'gapopen', 'qstart',
           'qend', 'sstart', 'send', 'evalue', 'bitscore']
df.columns = headers

## Filter above bitscore 100 and evalue > 1e-5
df_filtered = df[(df['evalue']<=1e-5) & (df['bitscore']>=100)]
df_filtered = df_filtered[df_filtered['pident']>=80]

In [3]:
def table_to_output(groupby_table):
    temp = groupby_table.reset_index()
    temp2 = temp[temp['True_Label']=="Phage"]
    if len(temp2)==0:
        False_Negative_Phage = Phage_Count
    else:
        Phage_Predicted = temp2.iloc[0][1]
        False_Negative_Phage = Phage_Count - Phage_Predicted
        
    
    temp3 = temp[temp['True_Label']=="Prophage"]
    if len(temp3)==0:
        False_Negative_Prophage = Prophage_Count
    else:
        Prophage_Predicted = temp3.iloc[0][1]
        False_Negative_Prophage = Prophage_Count - Prophage_Predicted
 
    return(False_Negative_Phage, False_Negative_Prophage)

In [20]:
## Score without prophages
def Score(FNP, data):
    
    data2 = data[data["True_Label"]!="Prophage"]
    y_true = [1 if x == "Phage" else 0 for x in data2['True_Label']]
    
    y_pred = [1]*len(data2)
    
    listofzeros = [0]*FNP
    listofones = [1]*FNP

    y_pred = y_pred + listofzeros
    y_true = y_true + listofones
    
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return(p, r, f1)

In [ ]:
df_filtered['Contig'] = ["_".join(x.split("_")[:-1]) for x in df_filtered['qseqid']]
x = df_filtered.groupby("Contig").count()
x = x[['evalue']]
x = x.reset_index()
x['True_Label'] = [y.split("_")[0] for y in x['Contig']]
y = x[["True_Label", "evalue"]]
y.columns = ['True_Label', 'Earth Virome Gene Hits']
y = y.sort_values(by="Earth Virome Gene Hits", ascending=False)

In [26]:
z = y[y['Earth Virome Gene Hits']>=1]
e = z.groupby("True_Label").count()
FNP, FNPr = table_to_output(e)
p, r, f1 = Score(FNP, FNPr, z)

#print("Scores if prophages are considered Bacteria")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

Precision: 0.1590909090909091
Recall: 0.8076923076923077
F1 Score: 0.26582278481012656


In [239]:
e

,Earth Virome Gene Hits
True_Label,
Bacteria,82
Eukaroyte,7
Phage,4


In [27]:
my_dict = {}
max_f1 = 0
for i in range(1,100):
    z = y[y['Earth Virome Gene Hits']>=i]
    e = z.groupby("True_Label").count()
    FNP, FNPr = table_to_output(e)
    p, r, f1 = Score(FNP, FNPr, z, Prophage=False)
    if f1 > max_f1:
        max_f1 = f1
        id_max = i


#print("Scores if prophages are considered Bacteria")
#print("Precision: " + str(p))
#print("Recall: " + str(r))
print("F1 Score: " + str(max_f1))
print(id_max)

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


F1 Score: 0.23880597014925373
6


In [6]:
def Score(FNP, FNPro, data, Prophage=True):
    if Prophage==True:
        y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
        y_pred = [1]*len(data)

        listofzeros = [0]*FNP
        listofones = [1]*FNP

        y_pred = y_pred + listofzeros
        y_true = y_true + listofones

    else:
        data['True_Label'] = data['True_Label'].replace(regex='Prophage', value="Phage")
        y_true = [1 if x == "Phage" else 0 for x in data['True_Label']]
        y_pred = [1]*len(data)
        listofzeros = [0]*FNP
        listofones = [1]*FNP

        y_pred = y_pred + listofzeros
        y_true = y_true + listofones

        listofzeros2 = [0]*FNPro
        listofones2 = [1]*FNPro

        y_pred = y_pred + listofzeros2
        y_true = y_true + listofones2

        
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return(p, r, f1)

In [46]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
input_file = "../Earth_Virome/Earth_Virome_Data/Zero_Inflated_Lognormal_Low_Complexity.txt"
#input_file = "../../Nano_Reads_Tools/Nano_Combined_Assemblies_Diamond.txt"
df = pd.read_csv(input_file, header=None, sep="\t")


df2 = pd.read_csv("../../Contig_Abundance_By_Class_Simulation.csv")
simulation = input_file.split("_")[-5].split("/")[1]

if simulation == 'Zero':
    complexity = input_file.split("_")[-2]
else:
    complexity = input_file.split("_")[-2]


df1 = df2[(df2.Simulation == simulation) & (df2.Complexity == complexity)]
try:
    Phage_Count = int(df1['Phage'])
except:
    Phage_Count = 0
try:
    Prophage_Count = int(df1['Prophage'])
except:
    Prophage_Count = 0
    
 

headers = ['qseqid', 'sseqid','pident','length', 'mismatch', 'gapopen', 'qstart',
           'qend', 'sstart', 'send', 'evalue', 'bitscore']
df.columns = headers

## Filter above bitscore 100 and evalue > 1e-5
df_filtered = df[(df['evalue']<=1e-5) & (df['bitscore']>=100)]
df_filtered = df_filtered[df_filtered['pident']>=80]

In [47]:
df_filtered['Contig'] = ["_".join(x.split("_")[:-1]) for x in df_filtered['qseqid']]
x = df_filtered.groupby("Contig").count()
x = x[['evalue']]
x = x.reset_index()
x['True_Label'] = [y.split("_")[0] for y in x['Contig']]
y = x[["True_Label", "evalue"]]
y.columns = ['True_Label', 'Earth Virome Gene Hits']
y = y.sort_values(by="Earth Virome Gene Hits", ascending=False)

In [48]:
my_dict = {}
z = y[y['Earth Virome Gene Hits']>=1]
e = z.groupby("True_Label").count()
FNP, FNPro = table_to_output(e)
p, r, f1 = Score(FNP, FNPro, z)
my_dict['Prophages_Bacteria'] = [p,r,f1]

p, r, f1 = Score(FNP, FNPro, z, Prophage=False)
my_dict['Prophages_Viruses'] = [p,r,f1]
### Save output
w = pd.DataFrame(my_dict.items(), columns=["Prophage Category",2])
w = w.set_index("Prophage Category")
w['Precision'] = [x[0] for x in w[2]]
w['Recall'] = [x[1] for x in w[2]]
w['F1 Score'] = [x[2] for x in w[2]]
w = w.drop(2, axis=1)


w['Simulation'] = "zero"
w['Complexity'] = "low"
w['Tool'] = 'Earth_Virome'
w['Parameter'] = "Full"
#w.to_csv("Earth_Virome_Output_without_prophages.csv")
w.to_csv("Earth_Virome_Output_ident.csv", header=False, mode='a')
#w

In [28]:
w

,Precision,Recall,F1 Score,Simulation,Complexity,Tool,Parameter
Prophage Category,,,,,,,
Prophages_Bacteria,0.055556,0.818182,0.104046,lognormal,low,Earth_Virome,Full
Prophages_Viruses,0.086420,0.212121,0.122807,lognormal,low,Earth_Virome,Full


In [25]:
z = y[y['Earth Virome Gene Hits']>=1]
e = z.groupby("True_Label").count()
FNP, FNPro = table_to_output(e)
p, r, f1 = Score(FNP, FNPro, z)

print("Scores if prophages are considered Bacteria")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

p, r, f1 = Score(FNP, FNPro, z, Prophage=False)
print("Scores if prophages are considered Viruses")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

Scores if prophages are considered Bacteria
Precision: 0.01042729230420492
Recall: 0.5
F1 Score: 0.02042855556789164
Scores if prophages are considered Viruses
Precision: 0.04813177679549662
Recall: 0.5
F1 Score: 0.08781059378984733


1088

In [11]:
z = y[y['Earth Virome Gene Hits']>=25]
e = z.groupby("True_Label").count()
FNP, FNPro = table_to_output(e)
p, r, f1 = Score(FNP, FNPro, z)
print("Scores if prophages are considered Bacteria")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

p, r, f1 = Score(FNP, FNPro, z, Prophage=False)
print("Scores if prophages are considered Viruses")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

Scores if prophages are considered Bacteria
Precision: 0.07720588235294118
Recall: 1.0
F1 Score: 0.14334470989761094
Scores if prophages are considered Viruses
Precision: 0.10661764705882353
Recall: 1.0
F1 Score: 0.1926910299003322


/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [8]:
q = y[y['Earth Virome Gene Hits']>1]
t = q.groupby("True_Label").count()
FNP, FNPro = table_to_output(t)
p, r, f1 = Score(FNP, FNPro, q)
print("Scores if prophages are considered Bacteria")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

p, r, f1 = Score(FNP, FNPro, q, Prophage=False)
print("Scores if prophages are considered Viruses")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

Scores if prophages are considered Bacteria
Precision: 0.009900990099009901
Recall: 1.0
F1 Score: 0.0196078431372549
Scores if prophages are considered Viruses
Precision: 0.0144014401440144
Recall: 1.0
F1 Score: 0.028393966282165038


/Users/stronglab2/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [9]:
z = y.groupby("True_Label").count()
FNP, FNPro = table_to_output(z)
## Prophage as bacteria
p, r, f1 = Score(FNP, FNPro, y)
print("Scores if prophages are considered Bacteria")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))

p, r, f1 = Score(FNP, FNPro, y, Prophage=False)
print("Scores if prophages are considered Viruses")
print("Precision: " + str(p))
print("Recall: " + str(r))
print("F1 Score: " + str(f1))


Scores if prophages are considered Bacteria
Precision: 0.00636604774535809
Recall: 1.0
F1 Score: 0.012651555086979443
Scores if prophages are considered Viruses
Precision: 0.016976127320954906
Recall: 1.0
F1 Score: 0.03338549817423056
